In [16]:

%matplotlib inline

import os
import torch
import trimesh
import numpy as np
import torch.nn as nn

if os.getcwd().split("\\")[-1] != "toytorch":
    os.chdir(os.path.join(os.getcwd(), "../"))

from deepSDF.config import Configuration
from deepSDF.synthesize import Synthesizer
from deepSDF.model import SDFdataset, SDFdecoder, SDFdecoderTrainer
from deepSDF.data.data_creator import DataCreator, DataCreatorHelper

In [ ]:
# Visualizing raw skyscrapers

path = Configuration.RAW_DATA_PATH
meshes = trimesh.Trimesh(vertices=[], faces=[])

for i, file in enumerate(os.listdir(path)):
    if file.endswith(".obj"):
        mesh = trimesh.load(os.path.join(path, file))

        if isinstance(mesh, trimesh.Scene):
            geo_list = []
            for g in mesh.geometry.values():
                geo_list.append(g)
            mesh = trimesh.util.concatenate(geo_list)

        mesh.fix_normals(multibody=True)
        
        meshes += mesh
        
# If you want to see the raw data, uncomment the following lines.

# scene = trimesh.Scene(meshes)
# scene.set_camera(angles=[90, 90, 90])
# scene.show()


In [ ]:
# Preprocessing data

data_creator = DataCreator(
    n_surface_sampling=Configuration.N_SURFACE_SAMPLING,
    n_bbox_sampling=Configuration.N_BBOX_SAMPLING,
    n_volume_sampling=Configuration.N_VOLUME_SAMPLING,
    raw_data_path=Configuration.RAW_DATA_PATH,
    save_path=Configuration.SAVE_DATA_PATH,
    translate_mode=DataCreatorHelper.CENTER_WITHOUT_Z,
    dynamic_sampling=False,
    is_debug_mode=False,
)

data_creator.create()

2024-03-31 13:10:21,929	INFO worker.py:1724 -- Started a local Ray instance.
Preprocessing:   0%|          | 0/15 [00:00<?, ?it/s]

mesh_vertices_count: 402002 n_total_sampling: 262144


Preprocessing:   7%|▋         | 1/15 [00:04<01:09,  4.94s/it]

mesh_vertices_count: 252401 n_total_sampling: 262144


Preprocessing:  13%|█▎        | 2/15 [00:08<00:56,  4.33s/it]

...

mesh_vertices_count: 655553 n_total_sampling: 262144


Preprocessing:  93%|█████████▎| 14/15 [01:26<00:07,  7.50s/it]

mesh_vertices_count: 520294 n_total_sampling: 262144


Preprocessing: 100%|██████████| 15/15 [01:32<00:00,  6.20s/it]


The function create took 208.49423742294312 seconds to run.


In [ ]:
# Training model

sdf_dataset = SDFdataset()
sdf_decoder = SDFdecoder(cls_nums=sdf_dataset.cls_nums)
sdf_trainer = SDFdecoderTrainer(
    sdf_dataset=sdf_dataset,
    sdf_decoder=sdf_decoder,
    is_debug_mode=False,
    seed=77777,
    pre_trained_path=r"deepSDF\runs\2024-03-24_12-40-26",
)

sdf_trainer.train()

CUDA status
  torch.cuda.is_available(): True
  DEVICE: cuda 

Seeds status:
  Seeds set for torch        : 77777
  Seeds set for torch on GPU : 77777
  Seeds set for numpy        : 77777
  Seeds set for random       : 77777 

Set all pre-trained states from deepSDF\runs\2024-03-24_12-40-26\states\all_states.pth 

Set all dataloaders
  TRAIN_DATASET_RATIO: 0.8
  VAL_DATASET_RATIO: 0.2 

Set all pre-trained optimizers 

Set all pre-trained weights 

best_loss: 0.13492946143469453 



c:\Users\park1\anaconda3\envs\pytorch-exercise\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: 141th epoch took 1019.885041475296 seconds


Epoch: 142th Train Loss: 0.13484037555925474, Val Loss: 0.13492944790535452
Epoch: 142th epoch took 997.0486843585968 seconds


Epoch: 143th epoch took 1022.3658022880554 seconds


...

Epoch: 146th epoch took 1048.487384557724 seconds


Epoch: 147th Train Loss: 0.13484034106265122, Val Loss: 0.1349294140651788
Epoch: 147th epoch took 1021.6076176166534 seconds


Epoch: 148th epoch took 1012.7693212032318 seconds


Epoch: 149th epoch took 999.1915583610535 seconds


Epoch: 150th Train Loss: 0.13484031601835036, Val Loss: 0.1349294129622649
Epoch: 150th epoch took 1017.3521234989166 seconds


In [ ]:
# Visualizing reconstructed skyscrapers from the trained latent codes

"""
    cls_dict = {
        0: 'bank_of_china',
        1: 'burj_al_arab',
        2: 'cctv_headquarter',
        3: 'china_zun',
        4: 'empire_state_building',
        5: 'hearst_tower',
        6: 'kingdom_centre',
        7: 'lotte_tower',
        8: 'mahanakhon',
        9: 'one_world_trade_center',
        10: 'shanghai_world_financial_center',
        11: 'taipei_101',
        12: 'the_gherkin',
        13: 'the_shard',
        14: 'transamerica_pyramid'
    }
"""

all_states = torch.load(r"deepSDF\runs\2024-03-24_12-40-26\states\all_states.pth")
cls_dict = all_states["cls_dict"]

sdf_decoder = SDFdecoder(cls_nums=max(cls_dict.keys()) + 1)
sdf_decoder.load_state_dict(all_states["model_d"])
sdf_decoder.latent_codes = nn.Parameter(all_states["latent_codes"])

sdf_trainer = SDFdecoderTrainer(
    sdf_dataset=None,
    sdf_decoder=sdf_decoder,
    seed=77777,
    is_reconstruct_mode=True,
)

os.makedirs(r"deepSDF\runs\2024-03-24_12-40-26\reconstructed_after_training", exist_ok=True)

mesh_interval = 1
meshes = trimesh.Trimesh(vertices=[], faces=[])
for cls_num in cls_dict.keys():
    
    obj_path = f"deepSDF\\runs\\2024-03-24_12-40-26\\reconstructed_after_training\\{cls_num}.obj"
    
    if os.path.exists(obj_path.replace(".obj", f"_{None}_{cls_dict[cls_num]}.obj")):
        continue

    sdf_trainer.reconstruct(
        sdf_decoder=sdf_decoder,
        cls_dict=cls_dict,
        obj_path=obj_path,
        epoch=None,
        cls_num=cls_num,
        map_z_to_y=True,
    )
    
    mesh = trimesh.load(obj_path.replace(".obj", f"_{None}_{cls_dict[cls_num]}.obj"))

    if isinstance(mesh, trimesh.Scene):
        geo_list = []
        for g in mesh.geometry.values():
            geo_list.append(g)
        mesh = trimesh.util.concatenate(geo_list)

    mesh.fix_normals(multibody=True)
    
    mesh.vertices += np.array([mesh_interval * cls_num, 0, 0])
    
    meshes += mesh
    
# If you want to see the skyscrapers generated by model, uncomment the following lines.

# scene = trimesh.Scene(meshes)
# scene.set_camera(angles=[90, 90, 90])
# scene.show()    

CUDA status
  torch.cuda.is_available(): True
  DEVICE: cuda 

Seeds status:
  Seeds set for torch        : 77777
  Seeds set for torch on GPU : 77777
  Seeds set for numpy        : 77777
  Seeds set for random       : 77777 



In [1]:
# Synthesizing trained data

import os

if os.getcwd().split("\\")[-1] != "toytorch":
    os.chdir(os.path.join(os.getcwd(), "../"))

import torch
import torch.nn as nn
import gc

torch.cuda.empty_cache()
gc.collect()

from deepSDF.model import SDFdecoder
from deepSDF.synthesize import Synthesizer


"""
    cls_dict = {
        0: 'bank_of_china',
        1: 'burj_al_arab',
        2: 'cctv_headquarter',
        3: 'china_zun',
        4: 'empire_state_building',
        5: 'hearst_tower',
        6: 'kingdom_centre',
        7: 'lotte_tower',
        8: 'mahanakhon',
        9: 'one_world_trade_center',
        10: 'shanghai_world_financial_center',
        11: 'taipei_101',
        12: 'the_gherkin',
        13: 'the_shard',
        14: 'transamerica_pyramid'
    }
"""

all_states = torch.load(r"deepSDF\runs\2024-03-24_12-40-26\states\all_states.pth")
cls_dict = all_states["cls_dict"]
cls_nums = max(cls_dict.keys()) + 1

sdf_decoder = SDFdecoder(cls_nums=cls_nums)
sdf_decoder.load_state_dict(all_states["model_d"])
sdf_decoder.latent_codes = nn.Parameter(all_states["latent_codes"])

synthesizer = Synthesizer()

In [ ]:
# Synthesizing trained data by interpolating two latent codes

import os
import time
import torch
import random
import numpy as np
from IPython.display import clear_output

synthesized_dir = r"deepSDF\runs\2024-03-24_12-40-26\synthesized"
synthesized_latent_codes_npz = "synthesized_latent_codes.npz"
synthesized_latent_codes_path = os.path.join(synthesized_dir, synthesized_latent_codes_npz)

os.makedirs(synthesized_dir, exist_ok=True)

synthesized_latent_codes = {
    "data": [
        {
            "name": i,
            "index": i,
            "interpolation_factor": None,
            "latent_code": list(latent_code.detach().cpu().numpy()),
        } for i, latent_code in enumerate(sdf_decoder.latent_codes)
    ] 
}

if os.path.exists(synthesized_latent_codes_path):
    synthesized_latent_codes = {
        "data": list(np.load(synthesized_latent_codes_path, allow_pickle=True)["synthesized_data"])
    }
        
while len([f for f in os.listdir(synthesized_dir) if f.endswith(".obj")]) < 135:
    
    print(
        f"len([f for f in os.listdir(synthesized_dir) if f.endswith('.obj')]): \
        {len([f for f in os.listdir(synthesized_dir) if f.endswith('.obj')])}"
    )
    
    data_to_sample = synthesized_latent_codes["data"]
    
    if random.Random(time.time()).random() < 0.5:
        data_to_sample = data_to_sample[:len(sdf_decoder.latent_codes)]
    
    data_1, data_2 = random.Random(time.time()).sample(data_to_sample, 2)
    
    latent_code_1 = data_1["latent_code"]
    latent_code_1 = torch.tensor(latent_code_1).to(sdf_decoder.latent_codes.device)
    latent_code_1_index = data_1["index"]
    
    latent_code_2 = data_2["latent_code"]
    latent_code_2 = torch.tensor(latent_code_2).to(sdf_decoder.latent_codes.device)
    latent_code_2_index = data_2["index"]
    
    random_interpolation_factor = round(0.25 + (0.75 - 0.25) * random.Random(time.time()).random(), 3)
    
    synthesized_latent_code = synthesizer.interpolate(
        latent_codes=[latent_code_1, latent_code_2],
        factors=[random_interpolation_factor]
    )
    
    name = f"{latent_code_1_index}__{latent_code_2_index}__{str(random_interpolation_factor).replace('.', '-')}.obj"
    save_name = os.path.join(synthesized_dir, name)
    
    if os.path.exists(save_name):
        continue
    
    synthesized_mesh = synthesizer.synthesize(
        sdf_decoder=sdf_decoder,
        latent_code=synthesized_latent_code,
        resolution=384,
        save_name=save_name,
        map_z_to_y=True,
        check_watertight=True,
    )

    synthesized_data = {
        "name": name,
        "index": len(synthesized_latent_codes["data"]),
        "interpolation_factor": random_interpolation_factor,
        "latent_code": list(synthesized_latent_code.detach().cpu().numpy()),
    }
    
    synthesized_latent_codes["data"].append(synthesized_data)
    
    np.savez(
        synthesized_latent_codes_path,
        synthesized_data=np.array(synthesized_latent_codes["data"]),
    )
    
    clear_output(wait=False)


In [ ]:
# Synthesizing trained data by adding and subtracting three latent codes

import os
import time
import torch
import random
import numpy as np
from IPython.display import clear_output

synthesized_dir = r"deepSDF\runs\2024-03-24_12-40-26\synthesized"
synthesized_latent_codes_npz = "synthesized_latent_codes_2.npz"
synthesized_latent_codes_path = os.path.join(synthesized_dir, synthesized_latent_codes_npz)

os.makedirs(synthesized_dir, exist_ok=True)

synthesized_latent_codes = {
    "data": [
        {
            "name": i,
            "index": i,
            "latent_code": list(latent_code.detach().cpu().numpy()),
        } for i, latent_code in enumerate(sdf_decoder.latent_codes)
    ] 
}

if os.path.exists(synthesized_latent_codes_path):
    synthesized_latent_codes = {
        "data": list(np.load(synthesized_latent_codes_path, allow_pickle=True)["synthesized_data"])
    }
    
        
while len([f for f in os.listdir(synthesized_dir) if f.endswith(".obj")]) < 300 - 15:
    
    print(
        f"len([f for f in os.listdir(synthesized_dir) if f.endswith('.obj')]): \
        {len([f for f in os.listdir(synthesized_dir) if f.endswith('.obj')])}"
    )
    
    data_to_sample = synthesized_latent_codes["data"]
    
    if random.Random(time.time()).random() < 0.5:
        data_to_sample = data_to_sample[:len(sdf_decoder.latent_codes)]
    
    random_data = random.Random(time.time()).sample(data_to_sample, 3)
    
    selected_indices = str(random_data[0]["index"])
    synthesized_latent_code = torch.tensor(random_data[0]["latent_code"]).to(sdf_decoder.latent_codes.device)
    for rdi, rd in enumerate(random_data[1:]):
        if rdi != len(random_data[1:]) - 1:
            synthesized_latent_code += torch.tensor(rd["latent_code"]).to(sdf_decoder.latent_codes.device)
        else:
            synthesized_latent_code -= torch.tensor(rd["latent_code"]).to(sdf_decoder.latent_codes.device)
            
        selected_indices += "__" + str(rd["index"])
    
    name = f"{selected_indices}.obj"
    save_name = os.path.join(synthesized_dir, name)
    
    print(save_name)
    
    if os.path.exists(save_name):
        continue
    
    synthesized_mesh = synthesizer.synthesize(
        sdf_decoder=sdf_decoder,
        latent_code=synthesized_latent_code,
        resolution=384,
        save_name=save_name,
        map_z_to_y=True,
        check_watertight=True,
    )

    synthesized_data = {
        "name": name,
        "index": len(synthesized_latent_codes["data"]),
        "latent_code": list(synthesized_latent_code.detach().cpu().numpy()),
    }
    
    synthesized_latent_codes["data"].append(synthesized_data)
    
    np.savez(
        synthesized_latent_codes_path,
        synthesized_data=np.array(synthesized_latent_codes["data"]),
    )
    
    clear_output(wait=False)
